# Import Library

In [1]:
import pandas as pd
from pythainlp import word_tokenize as tokenize
from pythainlp.util import *
from tqdm.auto import tqdm
import sys
from collections import Counter
sys.path.append('../Cython')
from Method import *
tqdm.pandas()

C:\Users\Alps\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load Data

In [2]:
df = pd.read_csv('../assets/SplitData/2021_10_24.csv')
df

,title,summary,link,published
0,เปิด 5 ประเด็นจาก 5 เวที GCNT Forum 2021รวมพลั...,เปิด 5 ประเด็นจาก 5 เวที GCNT Forum 2021รวมพลั...,https://thaipublica.org/2021/10/gcnt-forum-202...,2021_10_24
1,PTTGC ทุ่มงบ 1.65 แสนล้าน ตั้งเป้าลดการปล่อยก๊...,PTTGC ทุ่มงบ 1.65 แสนล้าน ประกาศแผนงานสู่เป้าห...,https://thaipublica.org/2021/10/pttgc-together...,2021_10_24
2,ASEAN Roundup เวียดนามเปิดรับวัคซีนพาสปอร์ต 72...,ASEAN Roundup ประจำวันที่ 17-23 ตุลาคม 2564 เว...,https://thaipublica.org/2021/10/asean-weekly-r...,2021_10_24
3,กองทุนน้ำมันเชื้อเพลิงติดกับดัก “ประชานิยม”,กองทุนน้ำมันเชื้อเพลิงติดกับดัก “ประชานิยม” คว...,https://thaipublica.org/2021/10/oil-fuel-fund-...,2021_10_24
4,วิกฤติศรัทธาของชาวฮินดูและไทยพุทธ,การปฏิรูปกิจกรรมพระพุทธศาสนาโดยการเพิกเฉยต่อวั...,https://thaipublica.org/2021/10/mano-laohavani...,2021_10_24
...,...,...,...,...
599,&quot;นิพนธ์&quot; สั่ง ผู้ว่าฯ เตือนประชาชน เ...,"""รมช.นิพนธ์"" กำชับ เจ้าหน้าที่รับมือสถานการณ์น...",https://www.thairath.co.th/news/politic/2226831,2021_10_24
600,รอบรั้วการตลาด : แม็คโครจัด Taste of Australia...,รอบรั้วการตลาด ประจำวันที่ 24 ต.ค. 64 ทีมข่าวไ...,https://www.thairath.co.th/business/market/222...,2021_10_24
601,จับแรงงานพม่า 120 คน ซ่อนตัวในป่าไทรโยค จะไปทำ...,แรงงานเถื่อนทะลัก ที่ จ.กาญจนบุรี ทหารสนธิกำลั...,https://www.thairath.co.th/news/local/central/...,2021_10_24
602,แอนชิลี สวยชนะใจกรรมการ ฟาดมง MUT2021,รอๆ กันมานาน ในที่สุดก็ได้นางงามคนใหม่กันแล้ว ...,https://www.thairath.co.th/entertain/news/2226591,2021_10_24


# Read Stop-Word Method

In [3]:
def read_stop_word(filename):
    text_file = open(f"../assets/StopWordList/{filename}.txt", "r", encoding='utf-8')
    content = text_file.read()
    dict_list = content.split("\n")
    text_file.close()
    return dict_list

In [4]:
newmm_stopword = read_stop_word("newmm_stopword_m10_th20")

In [5]:
attacut_stopword = read_stop_word("attacut_stopword_m10_th20")

# Word Break Method

In [6]:
NEWMM = "newmm"
ATTACUT = "attacut"
list_column = df.columns.values.tolist()
column_name = list_column[1]

def word_break(news,engine):
    segment = tokenize(str(news),keep_whitespace=False,engine=engine)
    return segment

In [7]:
atta_list = []
newmm_list = []

def wordBreak(df):
    l = len(df)
    for i in tqdm(range(l)):
        wb_list = word_break(df.iloc[i][column_name],ATTACUT)
        remove_sw = [word for word in wb_list  if word not in attacut_stopword]
        atta_list.append(remove_sw)
        wb_list = word_break(df.iloc[i][column_name],NEWMM)
        remove_sw = [word for word in wb_list  if word not in newmm_stopword]
        newmm_list.append(remove_sw)

wordBreak(df)

100%|██████████| 604/604 [00:28<00:00, 21.34it/s]


# LSC Method

In [8]:
def LCS(atta_list, newmm_list):
    l = len(atta_list)
    lcs_atta = []
    lcs_newmm = []
    for i in tqdm(range(l-1)):
        w1_atta = atta_list[i]
        len_w1_atta = len(w1_atta)
        w1_newmm = newmm_list[i]
        len_w1_newmm = len(w1_newmm)
        for j in range(i+1,l):
            w2_atta = atta_list[j]
            len_w2_atta = len(w2_atta)
            w2_newmm = newmm_list[j]
            len_w2_newmm = len(w2_newmm)
            lcs_atta_list,maxl_atta = [],0
            lcs_newmm_list,maxl_newmm = [],0
            if i != j:
                lcs_atta_list,maxl_atta = lcs(w1_atta,w2_atta,len_w1_atta,len_w2_atta)
                lcs_newmm_list,maxl_newmm = lcs(w1_newmm,w2_newmm,len_w1_newmm,len_w2_newmm)
                if lcs_atta_list != []:
                    lcs_atta.extend(lcs_atta_list)
                if lcs_newmm_list != []:
                    lcs_newmm.extend(lcs_newmm_list)

    lcs_atta_counter = Counter(lcs_atta)
    lcs_newmm_counter = Counter(lcs_newmm)
    return lcs_atta_counter, lcs_newmm_counter

In [9]:
def LCS_result(data):
    # print("result")
    df_result = pd.DataFrame(data.items(), columns=['LCS', 'round'])
    df_result['total_round'] = df_result['round'].sum()
    df_result.sort_values(['round'], ascending=False, inplace=True, ignore_index=True)
    return df_result

# Result

In [10]:
lcs_atta, lcs_newmm = LCS(atta_list, newmm_list)

100%|██████████| 603/603 [00:01<00:00, 392.60it/s]


In [11]:
result_newmm = LCS_result(lcs_newmm)
result_newmm[:20]

,LCS,round,total_round
0,แอนชิลีสก๊อตมิส,40,215
1,บาสปอป้อ,16,215
2,แอนแอนชิลีสก๊อตมิส,14,215
3,ผบภ.จว.,5,215
4,ความกดอากาศแผ่ปกคลุม,4,215
5,ขบวนอัญเชิญเกี้ยวประเพณี,4,215
6,ชีพตันตระวาณิชย์,3,215
7,ดำรงกิตติภัสร์,3,215
8,ขบวนอัญเชิญเกี้ยวประเพณีจุฬาฯ,3,215
9,ยุทธจรัสเสถียรมหาสารคามรองหัวหน้าแคนดิเดต,3,215


In [12]:
len(result_newmm)

101

In [13]:
trend = result_newmm[result_newmm["LCS"].str.contains("แอนชิลี")]
trend

,LCS,round,total_round
0,แอนชิลีสก๊อตมิส,40,215
2,แอนแอนชิลีสก๊อตมิส,14,215
10,แอนชิลีสก๊อตมิสลูกครึ่ง,2,215
24,แอนชิลีลูกครึ่ง,2,215
98,แอนชิลีสก๊อตมิสมงกุฎ,1,215


In [14]:
news = df[df["summary"].str.contains("แอนชิลี")]
news

,title,summary,link,published
40,“แอนชิลี สก๊อต-เคมมิส” ลูกครึ่งไทย-ออสเตรเลีย ...,“แอนชิลี สก๊อต เคมมิส” ลูกครึ่งไทย-ออสเตรเล […...,https://www.prachachat.net/spinoff/entertainme...,2021_10_24
42,รู้จัก “แอนชิลี สก๊อต-เคมมิส” Miss Universe Th...,“แอน” แอนชิลี สก๊อต-เคมมิส ลูกครึ่งไทย-ออสเตรเ...,https://www.prachachat.net/spinoff/entertainme...,2021_10_24
163,ส่อง 'Top 5' ตัวเต็ง Miss Universe Thailand 20...,การประกวดมิสยูนิเวิร์สไทยแลนด์ 2021 (Miss Univ...,https://www.naewna.com/lady/611006,2021_10_24
190,'แอนชิลี' สก๊อต-เคมมิส คว้ามงกุฎ Miss Universe...,'แอนชิลี' สก๊อต-เคมมิส คว้ามงกุฎ Miss Universe...,https://www.naewna.com/lady/611077,2021_10_24
368,สวยสง่า ‘แอนชิลี’ ลูกครึ่งไทย-ออสเตรเลีย คว้าม...,สวยสง่า ‘แอนชิลี’ ลูกครึ่งไทย-ออสเตรเลีย คว […...,https://www.matichon.co.th/lifestyle/social-wo...,2021_10_24
381,ทำความรู้จัก ‘แอนชิลี’ มิสยูนิเวิร์สไทยแลนด์คน...,ทำความรู้จัก ‘แอนชิลี’ มิสยูนิเวิร์สไทยแลนด […...,https://www.matichon.co.th/lifestyle/social-wo...,2021_10_24
425,“แอนชิลี สก๊อต-เคมมิส” ราศีจับม้วนเดียวจบ คว้า...,ตัดสินแล้ว “มิสยูนิเวิร์สไทยแลนด์ 2021” (Miss ...,https://bangkok-today.com/%e0%b9%81%e0%b8%ad%e...,2021_10_24
532,"""แอน-แอนชิลี สก๊อต-เคมมิส"" คว้ามงกุฎมิสยูนิเวิ...","ไม่หลุดโผ! ""แอน-แอนชิลี สก๊อต-เคมมิส"" คว้ามงกุ...",https://www.posttoday.com/ent/news/666323,2021_10_24
602,แอนชิลี สวยชนะใจกรรมการ ฟาดมง MUT2021,รอๆ กันมานาน ในที่สุดก็ได้นางงามคนใหม่กันแล้ว ...,https://www.thairath.co.th/entertain/news/2226591,2021_10_24


In [15]:
result_atta = LCS_result(lcs_atta)
result_atta.head(20)

,LCS,round,total_round
0,อุณหภูมิองศาเซลเซียส,16,121
1,ประกวดมิสยูนิเวิร์สไทยแลนด์,13,121
2,ขบวนอัญเชิญพระเกี้ยว,7,121
3,วินมอเตอร์ไซค์คิว,5,121
4,กดแผ่ปกคลุม,4,121
5,สวนดุสิตโพลมหาวิทยาลัยสวนดุสิต309,3,121
6,มิสยูนิเวิร์สไทยแลนด์MissUniverse,3,121
7,แชร์ “แม่มณีตุ๋น,2,121
8,ขนมทารีน่าโบเทส,2,121
9,กำชับจว.ตลิ่ง,2,121
